<a href="https://colab.research.google.com/github/HJiang437/tutorials/blob/master/GNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check the version of torch and cuda

In [1]:
import torch; print(torch.__version__)
import torch; print(torch.version.cuda)

1.9.0+cu102
10.2


# Download the dataset

In [4]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.7-cp37-cp37m-linux_x86_64.whl size=271433 sha256=dccaa7ba59fef5c59f86f78df046325157b9d50eeaae4cbeb997e66c331dd945
  Stored in directory: /root/.cache/pip/wheels/d4/47/a3/b454bae87fa5c1631826bc9b15285622675eac6baf4096b484
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
  Created wheel for torch-sparse: filename=torch_sparse-0.6.10-cp37-cp37m-linux_x86_64.whl size=391664 sha256=e0ec32e5bc286d965171e82a09cb5ee71322baa8b454dc73d913d2ffb9ef4815
  Stored in directory: /root/.cache/pip/wheels/4f/f7/04/45b5110d670b6e951625711b0cc237bad4a010fa4ef9e8b6d9
Successfully built torch-sparse
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+10.2.html
  Created wheel for torch-cluster: filename=torch_cluster-1.5.9-cp37-cp37m-linux_x86_64.whl size=327292 sha256=903517021b6a053a0f009644c33e0edddf5ff

# Nodes settings
1.   Nodes: the three cirles denoted by index numbered 0,1,2;
2.   Atrributes: the value if each node x1;
3.   Edges: the lines connecting the different nodes;

In [7]:
import torch
from torch_geometric.data import Data

edge_index=torch.tensor([[0,1,1,2],
                         [1,0,2,1]],dtype=torch.long) # Origin and Destination nodes
x=torch.tensor([[-1],[0],[1]],dtype=torch.float) 

data=Data(x=x,edge_index=edge_index)

In [9]:
print(data.num_nodes) # the number of nodes in the graph
print(data.num_edges) # the number of edges
print(data.num_edge_features) # number of attributes
print(data.contains_isolated_nodes()) # whether the graph contains any isolated nodes

3
4
0
False


# Training and evaluating simple GNNs using PyG

In [15]:
from torch_geometric.datasets import Planetoid
dataset=Planetoid(root='/tmp/Cora', name='Cora')

# Build Net with PyTorch

In [16]:
import torch 
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=GCNConv(dataset.num_node_features,16)
    self.conv2=GCNConv(16, dataset.num_classes)
  
  def forward(self, data):
    x, edge_index=data.x, data.edge_index
    x = self.conv1(x, edge_index) 
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

In [19]:
model=Net()
data=dataset[0]
optimizer=torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()

for epoch in range(200):
  optimizer.zero_grad()
  out=model(data)
  loss=F.nll_loss(out[data.train_mask],data.y[data.train_mask])
  loss.backward()
  optimizer.step()

# Evaluation

In [20]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8020
